In [ ]:
!pip install numpy
!pip install pandas

In [ ]:
import numpy as np
import math

# Implementation of a Node
# A node should be able to take an input vector (which is either the input or the previous layer)
# Sum the values from the previous node
# Pass the sum to an activation value
# Return the value as its own value

class node(object):
  def __init__(self, inputVec = [], weight = [], activation = None):
    # Number of Inputs
    self.inputVector = inputVec
    self.nodeWeights = weight
    self.actSelect = activation
    self.bias = np.random.rand()

    # Summation of Connected Notes
    self.nodeSum = self.summationNodes()

    # Activation function
    self.nodeVal = self.activation()

  # Parallelizable for all input values
  def summationNodes(self):
    sumOfVec = 0
    for (i, j) in zip(self.inputVector, self.nodeWeights):
        sumOfVec = sumOfVec + i * j
    return (sumOfVec + self.bias)

  def activation(self):
    if (self.actSelect == "sigmoid"):
      return (1 / (1 + pow(math.e, -self.nodeSum)))

    if (self.actSelect == "relu"):
      return (max(0.1*self.nodeSum, self.nodeSum))

In [ ]:
# For the mutli-layer perceptron
# We should first be able to store the input vector in an array
# Create N (user-defined number) nodes
# Allow users to add layers with custom nodes as a method of the class
# The output node automatically implements

class mlp(node):

  def __init__ (self):

    # We simulate the input layer here by storing each value
    # in the input as an element of a vector
    self.inputSize = None
    self.outputClass = []
    self.inputVector = [] # This is the vector we pass to the first hidden layer
    self.outputVal = None
    self.outputIndex = None

  def activate(self, inputVector, numClass):

    # We simulate the input layer here by storing each value
    # in the input as an element of a vector
    self.inputSize = len(inputVector)
    self.outputClass = numClass
    self.inputVector = inputVector # This is the vector we pass to the first hidden layer

    # Simulate hidden later
    layer1 = self.addLayer(numNodes = 10, prevLayer = self.inputVector)
    layer2 = self.addLayer(numNodes = 10, prevLayer = layer1)
    layer3 = self.addLayer(numNodes = 10, prevLayer = layer2)
    layer4 = self.addLayer(numNodes = 5, prevLayer = layer3)
    self.outputVal, self.outputIndex = self.outputLayer(layer4)

  # This intendeds to simulate the hidden layers
  def addLayer(self, numNodes = None, prevLayer = None):
    #weights = [np.random.rand() for i in range(len(inputVector))]

    if numNodes == None:
      numNodes = self.inputSize

    nextLayer = []
    for i in range(numNodes):
      # Create the nodes
      nextLayer.append(node(self.inputVector, [np.random.rand() for j in range(len(self.inputVector))], activation='relu'))

    outputValues = []
    for i in nextLayer:
      i.summationNodes()
      i.activation()
      outputValues.append(i.nodeVal)

    return outputValues

  def outputLayer(self, lastHidden):
    outLayer = []
    for i in range(self.outputClass):
      outLayer.append(node(lastHidden, [np.random.rand() for j in range(len(self.inputVector))], activation='sigmoid'))

    endVals = []
    for i in range(self.outputClass):
      endVals.append(outLayer[i].nodeVal)
      #print("Class ", i, " : ", outLayer[i].nodeVal)

    return max(endVals), endVals.index(max(endVals))

  def printResult(self):
    print("Predicted Class: ", self. outputIndex)

  def getPrediction(self):
    return self.outputIndex

# Sample with the Iris Dataset

In [ ]:
# Loading the IRIS Dataset for Classification
!pip install ucimlrepo
import pandas as pd
import numpy as np
from ucimlrepo import fetch_ucirepo

# fetch dataset
iris = fetch_ucirepo(id=53)

# data (as pandas dataframes)
df_X = iris.data.features
df_y = iris.data.targets

df_X = pd.DataFrame(df_X)
#df_X.head()

df_y = pd.DataFrame(df_y)
#df_y.head()

In [ ]:
# Perform some data wrangling/preparation

# Check info on the attributes
df_X.info() # All values are same type, check if any missing values
df_X.isnull().sum() # To check if there are any null values

# Checking the output
df_y.info()

# Object Types must be converted
df_y_original = df_y.copy() # Store the df somewhere before modifying

df_y.value_counts()

# Reveals Three Classes
# Map classes to corresponding numerical values

outMap = {'Iris-setosa':0, 'Iris-versicolor':1, 'Iris-virginica':2}
df_y.replace(outMap, inplace=True)

print(df_y)
df_y.value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal length  150 non-null    float64
 1   sepal width   150 non-null    float64
 2   petal length  150 non-null    float64
 3   petal width   150 non-null    float64
dtypes: float64(4)
memory usage: 4.8 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   class   150 non-null    object
dtypes: object(1)
memory usage: 1.3+ KB
     class
0        0
1        0
2        0
3        0
4        0
..     ...
145      2
146      2
147      2
148      2
149      2

[150 rows x 1 columns]


/tmp/ipykernel_2433/4190924181.py:19: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_y.replace(outMap, inplace=True)


class
0        50
1        50
2        50
Name: count, dtype: int64

# Test out the CPU Model

In [ ]:
def testModel(model, inVector, appendList = []):
  for i in inVector:
    model.activate(i, 3)
    appendList.append(model.getPrediction())

In [ ]:
predictedVals = []
model = mlp()

testModel(model, np.array(df_X), predictedVals)

In [ ]:
trueOutput = np.array(df_y)

correctPreds, totalPreds = 0, len(trueOutput)
for i in range(totalPreds):
  if predictedVals[i] == trueOutput[i]:
    correctPreds = correctPreds + 1

print("Correct Predictions / Total Predictions: {} / {} .".format(correctPreds, totalPreds))
print("Custom Feedforward Neural Network is: {} % Accurate".format(round(correctPreds/totalPreds * 100), 2))

Correct Predictions / Total Predictions: 51 / 150 .
Custom Feedforward Neural Network is: 34 % Accurate


# Implementation using CuPy

In [ ]:
# For implementation with CuPY
import cupy as cp

In [ ]:
nodeSum_kernel = cp.RawKernel(r'''
extern "C" __global__
void nodeSum_kernel(const float* inputVector, const float* layerWeights, float* outputNode) {
    int tid = blockDim.x * blockIdx.x + threadIdx.x;

    int temp = 0;
    for (int i = 0; i < blockDim.x; i++){
      temp = temp + inputVector[i] * layerWeights[i];
    }

    outputNode = temp;
}
''', 'nodeSum_kernel')

class MLPcupy():

  def __init__ (self):
    # We simulate the input layer here by storing each value
    # in the input as an element of a vector
    self.inputSize = None
    self.outputClass = []
    self.inputVector = [] # This is the vector we pass to the first hidden layer
    self.outputVal = None
    self.outputIndex = None

  def activate(self, inputVector, numClass):

    # We simulate the input layer here by storing each value
    # in the input as an element of a vector
    self.inputSize = len(inputVector)
    self.outputClass = numClass
    self.inputVector = inputVector # This is the vector we pass to the first hidden layer

    # Simulate hidden later
    layer1 = self.addLayer(numNodes = 10, prevLayer = self.inputVector)
    print(layer1)
    layer2 = self.addLayer(numNodes = 10, prevLayer = layer1)
    print(layer2)
    layer3 = self.addLayer(numNodes = 10, prevLayer = layer2)
    print(layer3)
    layer4 = self.addLayer(numNodes = 5, prevLayer = layer3)
    print(layer4)
    self.outputIndex = self.outputLayer(layer4)



  # This intendeds to simulate the hidden layers
  def addLayer(self, numNodes = None, prevLayer = None):

    if numNodes == None:
      print("Indicate number of Nodes.")
      return

    # This will be where we store the new values
    newLayer = cp.zeros(numNodes)

    # Generate the weights
    # weights = [cp.random.rand() for i in range(len(prevLayer))]
    weights = []
    for i in range(numNodes):
      weights.append([cp.random.rand() for i in range(len(prevLayer))])

    threads_per_block = 512
    size = numNodes
    grid_size = (int(math.ceil(size / threads_per_block)), 1, 1)
    block_size = (threads_per_block, 1, 1)

    for i in newLayer:
    # add_kernel(grid_size, block_size, (x1, x2, y, size))
    # nodeSum_kernel(const float* inputVector, const float* layerWeights, float* outputNode, const float* bias)
      nodeSum_kernel(grid_size, block_size, (cp.array(prevLayer), (cp.array([cp.random.rand() for i in range(len(prevLayer))])), i))

    cp.sum()

    return newLayer

  def outputLayer(self, lastHidden):
    outLayer = []
    for i in range(self.outputClass):
      outLayer.append(node(lastHidden, [cp.random.rand() for j in range(len(self.inputVector))], activation='sigmoid'))

    endVals = []
    for i in range(self.outputClass):
      endVals.append(outLayer[i].nodeVal)
      #print("Class ", i, " : ", outLayer[i].nodeVal)

    return endVals.index(max(endVals))

  def printResult(self):
    print("Predicted Class: ", self. outputIndex)

  def getPrediction(self):
    return self.outputIndex

Testing